In [1]:
import os
import random
import numpy as np

from collections import namedtuple

from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


# TREC - Question Answering (multi-class)

In [2]:
# TREC - Question Answering
# http://cogcomp.cs.illinois.edu/Data/QA/QC/

TREC_Question = namedtuple("TREC_Question", "label question")

trec_train = set()
trec_test = set()

for filename in os.listdir("TREC/"):
    with open("TREC/"+filename,'r', encoding='latin_1') as f_input:
        for line in f_input:
            label, question = line.split(' ', 1)
            label = label.split(':')[0]
            question = TREC_Question(label, question.strip())
            if filename=='TREC_10.label':
                trec_test.add(question)
            else:
                trec_train.add(question)

In [3]:
print("Train Samples: {}".format(len(trec_train)))
print("Test Samples : {}".format(len(trec_test)))
print("Labels       : {}".format({x.label for x in trec_train}))

Train Samples: 5381
Test Samples : 500
Labels       : {'ABBR', 'NUM', 'ENTY', 'LOC', 'DESC', 'HUM'}


In [4]:
# built two lists with sentences and labels
questions_train = [x.question for x in trec_train]
labels_train = [x.label for x in trec_train]

# convert list of tokens/words to indexes
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions_train)
sequences_train = tokenizer.texts_to_sequences(questions_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# get the max sentence lenght, needed for padding
max_input_lenght = max([len(x) for x in sequences_train])
print("Max. sequence lenght: ", max_input_lenght)

# pad all the sequences of indexes to the 'max_input_lenght'
data_train = pad_sequences(sequences_train, maxlen=max_input_lenght, padding='post', truncating='post')

# Encode the labels, each must be a vector with dim = num. of possible labels
le = LabelEncoder()
le.fit(labels_train)
labels_encoded_train = le.transform(labels_train)
categorical_labels_train = to_categorical(labels_encoded_train, num_classes=None)
print('Shape of train data tensor:', data_train.shape)
print('Shape of train label tensor:', categorical_labels_train.shape)

Found 8461 unique tokens.
Max. sequence lenght:  33
Shape of train data tensor: (5381, 33)
Shape of train label tensor: (5381, 6)


## TREC: test data

In [5]:
# pre-process test data
questions_test = [x.question for x in trec_test]
y_test = [x.label for x in trec_test]
sequences_test = tokenizer.texts_to_sequences(questions_test)
x_test = pad_sequences(sequences_test, maxlen=max_input_lenght)

labels_encoded_test = le.transform(y_test)
categorical_labels_test = to_categorical(labels_encoded_test, num_classes=None)
print('Shape of test data tensor:', x_test.shape)
print('Shape of test labels tensor:', categorical_labels_test.shape)

Shape of test data tensor: (500, 33)
Shape of test labels tensor: (500, 6)


In [6]:
from convnets_utils import *

# CNN with random word embeddings

In [7]:
model_1 = get_cnn_rand(300, len(word_index)+1, max_input_lenght, 6)

In [8]:
history = model_1.fit(x=data_train, y=categorical_labels_train, batch_size=50, epochs=10)

Epoch 1/10
5381/5381 [==============================] - 8s - loss: 0.4278 - acc: 0.8275     
Epoch 2/10
5381/5381 [==============================] - 9s - loss: 0.3234 - acc: 0.8676     
Epoch 3/10
5381/5381 [==============================] - 9s - loss: 0.2484 - acc: 0.8983     
Epoch 4/10
5381/5381 [==============================] - 9s - loss: 0.1980 - acc: 0.9200     - ETA: 0s - loss: 0.1981 - acc: 0.91
Epoch 5/10
5381/5381 [==============================] - 11s - loss: 0.1560 - acc: 0.9408    
Epoch 6/10
5381/5381 [==============================] - 9s - loss: 0.1252 - acc: 0.9528     
Epoch 7/10
5381/5381 [==============================] - 9s - loss: 0.0962 - acc: 0.9664     
Epoch 8/10
5381/5381 [==============================] - 9s - loss: 0.0751 - acc: 0.9753     
Epoch 9/10
5381/5381 [==============================] - 9s - loss: 0.0570 - acc: 0.9825     
Epoch 10/10
5381/5381 [==============================] - 9s - loss: 0.0479 - acc: 0.9853     


In [10]:
loss, accuracy = model_1.evaluate(x_test, categorical_labels_test, verbose=0)
accuracy

0.75866664552688601

In [11]:
raw_predictions = model_1.predict(x_test)
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test, le.inverse_transform(class_predictions)))

             precision    recall  f1-score   support

       ABBR       0.00      0.00      0.00         9
       DESC       0.28      1.00      0.43       138
       ENTY       0.00      0.00      0.00        94
        HUM       0.00      0.00      0.00        65
        LOC       0.00      0.00      0.00        81
        NUM       0.00      0.00      0.00       113

avg / total       0.08      0.28      0.12       500



/Users/dsbatista/virtual_envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# CNN with pre-trained static word embeddings

In [13]:
embeddings_index = load_fasttext_embeddings()
embeddings_matrix = create_embeddings_matrix(embeddings_index, word_index, 100)
embedding_layer_static = get_embeddings_layer(embeddings_matrix, 'embedding_layer_static', max_input_lenght, trainable=False)
model_2 = get_cnn_pre_trained_embeddings(embedding_layer_static, max_input_lenght, 6)

Found 400000 word vectors.
Matrix shape: (8462, 100)


In [15]:
history = model_2.fit(x=data_train, y=categorical_labels_train, batch_size=50, epochs=10)

Epoch 1/10
5381/5381 [==============================] - 1s - loss: 0.3811 - acc: 0.8440     
Epoch 2/10
5381/5381 [==============================] - 1s - loss: 0.2569 - acc: 0.8973     
Epoch 3/10
5381/5381 [==============================] - 1s - loss: 0.2083 - acc: 0.9152     
Epoch 4/10
5381/5381 [==============================] - 1s - loss: 0.1735 - acc: 0.9315     
Epoch 5/10
5381/5381 [==============================] - 1s - loss: 0.1450 - acc: 0.9442     
Epoch 6/10
5381/5381 [==============================] - 1s - loss: 0.1158 - acc: 0.9594     
Epoch 7/10
5381/5381 [==============================] - 1s - loss: 0.0905 - acc: 0.9714     
Epoch 8/10
5381/5381 [==============================] - 1s - loss: 0.0705 - acc: 0.9807     
Epoch 9/10
5381/5381 [==============================] - 1s - loss: 0.0525 - acc: 0.9881     
Epoch 10/10
5381/5381 [==============================] - 1s - loss: 0.0392 - acc: 0.9933     


In [16]:
loss, accuracy = model_2.evaluate(x_test, categorical_labels_test, verbose=0)
accuracy

0.76133331680297855

In [17]:
raw_predictions = model_2.predict(x_test)
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test, le.inverse_transform(class_predictions)))

             precision    recall  f1-score   support

       ABBR       0.00      0.00      0.00         9
       DESC       0.28      1.00      0.43       138
       ENTY       0.00      0.00      0.00        94
        HUM       0.00      0.00      0.00        65
        LOC       0.00      0.00      0.00        81
        NUM       0.00      0.00      0.00       113

avg / total       0.08      0.28      0.12       500



/Users/dsbatista/virtual_envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# CNN with pre-trained dynamic word embeddings

In [18]:
embedding_layer_dynamic = get_embeddings_layer(embeddings_matrix, 'embedding_layer_dynamic', max_input_lenght, trainable=True)
model_3 = get_cnn_pre_trained_embeddings(embedding_layer_dynamic, max_input_lenght, 6)

In [19]:
history = model_3.fit(x=data_train, y=categorical_labels_train, batch_size=50, epochs=10)

Epoch 1/10
5381/5381 [==============================] - 3s - loss: 0.3718 - acc: 0.8478     
Epoch 2/10
5381/5381 [==============================] - 3s - loss: 0.2277 - acc: 0.9084     
Epoch 3/10
5381/5381 [==============================] - 3s - loss: 0.1572 - acc: 0.9387     
Epoch 4/10
5381/5381 [==============================] - 3s - loss: 0.1059 - acc: 0.9617     
Epoch 5/10
5381/5381 [==============================] - 3s - loss: 0.0637 - acc: 0.9808     
Epoch 6/10
5381/5381 [==============================] - 3s - loss: 0.0368 - acc: 0.9904     
Epoch 7/10
5381/5381 [==============================] - 3s - loss: 0.0204 - acc: 0.9958     
Epoch 8/10
5381/5381 [==============================] - 3s - loss: 0.0113 - acc: 0.9984     
Epoch 9/10
5381/5381 [==============================] - 3s - loss: 0.0064 - acc: 0.9993     
Epoch 10/10
5381/5381 [==============================] - 3s - loss: 0.0040 - acc: 0.9998     


In [20]:
loss, accuracy = model_3.evaluate(x_test, categorical_labels_test, verbose=0)
accuracy

0.75866664552688601

In [21]:
raw_predictions = model_3.predict(x_test)
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test, le.inverse_transform(class_predictions)))

             precision    recall  f1-score   support

       ABBR       0.00      0.00      0.00         9
       DESC       0.28      1.00      0.43       138
       ENTY       0.00      0.00      0.00        94
        HUM       0.00      0.00      0.00        65
        LOC       0.00      0.00      0.00        81
        NUM       0.00      0.00      0.00       113

avg / total       0.08      0.28      0.12       500



/Users/dsbatista/virtual_envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# CNN multichanell with pre-trained dynamic and static word embeddings

In [22]:
model_4 = get_cnn_multichannel(embedding_layer_static, embedding_layer_dynamic, max_input_lenght, 6)

In [25]:
history = model_4.fit(x=[data_train, data_train], y=categorical_labels_train, batch_size=50, epochs=10)

Epoch 1/10
5381/5381 [==============================] - 4s - loss: 0.0296 - acc: 0.9925     
Epoch 2/10
5381/5381 [==============================] - 4s - loss: 0.0145 - acc: 0.9974     
Epoch 3/10
5381/5381 [==============================] - 4s - loss: 0.0072 - acc: 0.9991     
Epoch 4/10
5381/5381 [==============================] - 4s - loss: 0.0037 - acc: 0.9998     
Epoch 5/10
5381/5381 [==============================] - 5s - loss: 0.0021 - acc: 1.0000     
Epoch 6/10
5381/5381 [==============================] - 5s - loss: 0.0014 - acc: 1.0000     
Epoch 7/10
5381/5381 [==============================] - 5s - loss: 9.4511e-04 - acc: 1.0000     
Epoch 8/10
5381/5381 [==============================] - 5s - loss: 7.0011e-04 - acc: 1.0000     
Epoch 9/10
5381/5381 [==============================] - 5s - loss: 5.1368e-04 - acc: 1.0000     
Epoch 10/10
5381/5381 [==============================] - 5s - loss: 3.9920e-04 - acc: 1.0000     


In [29]:
loss, accuracy = model_4.evaluate(x=[x_test,x_test], y=categorical_labels_test, verbose=0)
accuracy

0.7589999799728393

In [30]:
raw_predictions = model_4.predict([x_test, x_test])
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test, le.inverse_transform(class_predictions)))

             precision    recall  f1-score   support

       ABBR       0.00      0.00      0.00         9
       DESC       0.28      1.00      0.43       138
       ENTY       0.00      0.00      0.00        94
        HUM       0.00      0.00      0.00        65
        LOC       0.00      0.00      0.00        81
        NUM       0.00      0.00      0.00       113

avg / total       0.08      0.28      0.12       500



/Users/dsbatista/virtual_envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
